In [1]:
from pathlib import Path

import altair as alt
import pandas as pd


In [2]:
frames = []
for jsonl in Path("results").glob("*.jsonl"):
    frame = pd.read_json(jsonl, lines=True)
    frame["machine"] = jsonl.stem
    frames.append(frame)

df = pd.concat(frames)

df["cores"] = df["threads"] * df["procs"]
df["parallelism"] = df.apply(lambda row: f"{row.procs}x{row.threads}", axis=1)

fenics= df[df.kind=="fenics"]
mumps = df[df.kind=="mumps"]
len(df)


489

In [3]:
from IPython.display import display

for machine, machine_df in mumps.groupby("machine"):
    for size, size_df in machine_df.groupby("size"):
        # filter out oversubscribe
        # size_df = size_df[size_df["procs"] * size_df["threads"] < 10]
        chart = alt.hconcat(*[
            alt.Chart(size_df).mark_point()
            .transform_calculate(
            env_threads = "datum.env + ' ' + datum.threads + 't'"
        ).encode(
            x=alt.X('env_threads:N', title="build + n threads"),
            y=y,
            color='env:N',
            shape='threads:N',
            # column='procs:N',
                tooltip=[y, "env", "threads", "procs"],
            ) for y in ("overall", "analyze",  "factorize", "solve")
        ], title=f"serial mumps test, {size=}, {machine=}")
        display(chart)


alt.HConcatChart(...)

alt.HConcatChart(...)

In [4]:
for machine, machine_df in fenics.groupby("machine"):
    for size, size_df in machine_df.groupby("size"):
        for cores, cores_df in size_df.groupby("cores"):
            # filter out oversubscribe
            # size_df = size_df[size_df["procs"] * size_df["threads"] < 10]
            chart = alt.hconcat(
                *[
                    alt.Chart(cores_df)
                    .mark_point()
                    .transform_calculate(env_threads="datum.env + ' ' + datum.parallelism")
                    .encode(
                        x=alt.X("env_threads:N", title="build + (procs x threads)"),
                        y=y,
                        color="env:N",
                        shape=alt.Shape("parallelism:N", title="procs x threads"),
                        # size='procs:N',
                        tooltip=[y, "env", "threads", "procs"],
                    )
                    for y in ("overall", "analyze", "factorize", "solve")
                ],
                title=f"fenics test {cores=}, {size=} {machine=!s}",
            )
            display(chart)


alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

In [5]:
for machine, machine_df in fenics.groupby("machine"):
    for size, size_df in machine_df.groupby("size"):
        # filter out oversubscribe
        # size_df = size_df[size_df["procs"] * size_df["threads"] < 10]
        chart = (
            alt.Chart(size_df, title=f"fenics test {size=} {machine=!s}")
            .mark_point()
            .transform_calculate(env_threads="datum.env + ' ' + datum.threads + 't'")
            .encode(
                x=alt.X("env_threads:N", title="build + (threads)"),
                y="factorize",
                color="env:N",
                shape=alt.Shape("parallelism:N", title="procs x threads"),
                column="procs:N",
                tooltip=["factorize", "env", "threads", "procs"],
            )
        )
        display(chart)


alt.Chart(...)

alt.Chart(...)